# K-Nearest Neighbors Algorithim:  Python Class Implementation


In [1]:
# import libraries
import requests
from contextlib import closing
import csv
import pandas as pd
from random import seed
from scipy.spatial.distance import pdist
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier


In [4]:
file_name = "IRIS.csv"
cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
dataframe = pd.read_csv(file_name, names=cols, skiprows=1)
dataset = pd.read_csv(file_name, names=cols, skiprows=1).drop('class', axis=1)
print(dataframe.head())
print(dataset.head())


   sepal_length  sepal_width  petal_length  petal_width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa
   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2


In [6]:
dataframe.describe()


,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [7]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [9]:
dataframe['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [17]:
class KNN:

    def __init__(self, dataframe, dataset):
        self.dataframe = dataframe
        self.dataset = dataset  
  
    col_class = dataframe['class']

    # Data Wrangling
    def string_to_float(self, dataset):
        """
        convert string column values to floats;
        not needed -> values are floats   
        """
        pass


    def ints_to_floats(self, dataset):
        """
        convert integers values to floats;
        not needed -> values are floats  
        """
        pass
    
    
    def create_unique_class_id(self, dataframe, col_class):
        """
        identify unique classification values; 
        encode column values as integer labels while 
        keeping human readable format too
        """
        cat = pd.Categorical(['1', '2', '3'], categories=['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
        self.class_val_labels, self.class_uniques = pd.factorize(cat)

        return self.class_val_labels, self.class_uniques    

   
    # Pre-Processing
    def min_max(self, dataset):
        """
        find the minimum and maximum values for each column;
        create a list of these values
        """
        # create empty list
        minmax = []
        for i in range(len(self.dataset):
            #col = [row[i] for row in self.dataset]
            minimum = row[i].min(self.dataset)   #min(col)
            maximum = row[i].max(self.dataset)  #max(col) 
            minmax.append([minimum, maximum])

        return self.minmax#, self.col


    def normalize(self, dataset, minmax):
        """
        rescale dataset columns to range 0-1;
        must be called after train/test split to avoid leakage/bias;
        returns ndarry of shape(n_samples, n_features) transformed data
        """
        for row in dataset:
          for i in range(len(row)):
            row[i] = (row[i] - self.minmax[i][0]) / (self.minmax[i][1] - self.minmax[i][0])     
    

    # def calculate_ed(self, v1, v2):
    #     """
    #     calculate euclidean distance between vectors
    #     """
    #     # convert to numpy arrays (just in case)
    #     v1, v2 = np.array(v1), np.array(v2)

    #     # set initial distance to zero
    #     distance = 0

    #     # iterate over each dimension of the vectors
    #     for i in range(len(v1) - 1):
    #         # increment distance
    #         distance += (v1[i] - v2[i]) **2

    #     # return the square root
    #     return np.sqrt(distance)


    def train_test_k_fold_split(self):
        """
        split dataset into k folds
        """
        pass

    
    # Implement K-NN Algorithm
    def get_neighbors(self):
        """
        locate most similar neighbors
        """
        pass
    

    def predict_class(self):
        """
        make a prediction with neighbors
        """
        pass
    

    def k_nn(self):
        """
        K-NN algorithm
        """
        pass

    
    # Evaluate K-NN Algorithm
    def accuracy_metrics(self):
        """
        calculate accuracy percentage
        """
        pass
    

    def cross_val_metric(self):
        """
        evaluate algorithm using cross validation 
        """
        pass


if __name__ == '__main__':
    # Test KNN class initializtion
    knn = KNN(dataframe, dataset)
    print(dataframe.head())
    print(dataframe.tail())
    dataframe.info()
    print(dataset)

    # test create_unigue_class_id method
    knn.create_unique_class_id(dataframe, "class")
    print(knn.class_val_labels)

    # test min_max method
    knn.min_max(dataset)
    print(knn.minmax)

    # test nortmalize method
    # knn.normalize(dataset, minmax)
    # print()

    # test train_test_k_fold_split method

    # test get_neighbors method

    # test predict_class method

    # test k_nn method

    # test accuracy_metrics method

    # test cross_val_metric method

     

SyntaxError: ignored